In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls './drive/My Drive/Colab Notebooks/Spark/Data'

part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00001-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00002-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00003-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00004-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00005-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00006-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00007-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00008-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00009-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00010-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part-00011-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json
part

In [ ]:
fpath = '/content/drive/My Drive/Colab Notebooks/Spark/Data'

In [ ]:
static = spark.read.json('/content/drive/My Drive/Colab Notebooks/Spark/Data/part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json')

In [ ]:
static.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [ ]:
streamdf = spark.readStream.option("MaxFilesPerTrigger",1).schema(static.schema).json(fpath)

In [ ]:
actCount = streamdf.groupBy("gt").count()

In [ ]:
actQuery = actCount.writeStream.queryName("acount").format("memory").outputMode("complete").start()

In [ ]:
spark.streams.active

In [ ]:
from time import sleep
for x in range(5):
  spark.sql("SELECT * FROM acount").show()
  sleep(1)

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10452|
|       sit|12310|
|     stand|11385|
|      walk|13256|
|      bike|10797|
|stairsdown| 9365|
|      null|10447|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10452|
|       sit|12310|
|     stand|11385|
|      walk|13256|
|      bike|10797|
|stairsdown| 9365|
|      null|10447|
+----------+-----+



In [ ]:
spark.sql("select * from acount limit 10").show()

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|20908|
|       sit|24617|
|     stand|22772|
|      walk|26512|
|      bike|21594|
|stairsdown|18727|
|      null|20894|
+----------+-----+



In [ ]:
spark.streams.active

In [ ]:
static.select("*").take(10)

[Row(Arrival_Time=1424686735175, Creation_Time=1424686733176178965, Device='nexus4_1', Index=35, Model='nexus4', User='g', gt='stand', x=0.0014038086, y=0.00050354, z=-0.0124053955),
 Row(Arrival_Time=1424686735378, Creation_Time=1424686733382813486, Device='nexus4_1', Index=76, Model='nexus4', User='g', gt='stand', x=-0.0039367676, y=0.026138306, z=-0.01133728),
 Row(Arrival_Time=1424686735577, Creation_Time=1424686733579072031, Device='nexus4_1', Index=115, Model='nexus4', User='g', gt='stand', x=0.003540039, y=-0.034744263, z=-0.019882202),
 Row(Arrival_Time=1424686735779, Creation_Time=1424688581834321412, Device='nexus4_2', Index=163, Model='nexus4', User='g', gt='stand', x=0.002822876, y=0.005584717, z=0.017318726),
 Row(Arrival_Time=1424686735982, Creation_Time=1424688582035859498, Device='nexus4_2', Index=203, Model='nexus4', User='g', gt='stand', x=0.0017547607, y=-0.018981934, z=-0.022201538),
 Row(Arrival_Time=1424686736186, Creation_Time=1424686734188508066, Device='nexus4_

In [ ]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [ ]:
from pyspark.sql.functions import expr

strans = streamdf.withColumn("stairs", expr("gt like '%stairs$'"))

In [ ]:
actQuery.stop()

In [ ]:
spark.streams.active

[]

In [ ]:
atrans = streamdf.withColumn("stairs", expr("gt like '%stairs$'")).where("gt is not null").select("gt", "model", "arrival_time", "creation_time", "stairs")

In [ ]:
atrm = atrans.writeStream.queryName("atransfm").format("memory").outputMode("append").start()

In [ ]:
spark.sql("select * from atransfm").take(10)

[]

In [ ]:
atrm.stop()

In [ ]:
simpleTransform = streamdf.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time")\
.writeStream\
.queryName("simple_transform")\
.format("memory")\
.outputMode("append")\
.start()

In [ ]:
simpleTransform.stop()

In [ ]:
spltrans = streamdf.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time")\
.writeStream\
.queryName("sformTable")\
.format("memory")\
.outputMode("append")\
.start()

In [ ]:
spark.sql("select * from sformTable").show()

+--------+------+-------------+-------------------+
|      gt| model| arrival_time|      creation_time|
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983725|1424689829780924966|
|stairsup|nexus4|1424687984011|1424687982020381820|
|stairsup|nexus4|1424687984413|1424687982421559529|
|stairsup|nexus4|1424687984818|1424687982827412800|
|stairsup|nexus4|1424687985223|1424687983227571661|
|stairsup|nexus4|1424687985623|1424687983630373175|
|stairsup|nexus4|1424687986026|1424687984033235724|
|stairsup|nexus4|1424687986428|1424687984435884650|
|stairsup|nexus4|1424687986833|1424687984840816323|
|stairsup|nexus4|1424687987240|1424687985246761147|
|stairsup|nexus4|1424687987635|1424687985644600852|
|stairsup|nexus4|1424687987994|1424689834040863906|
|stairsup|nexus4|1424687988200|1424687986208748802|
|stairsup|nexus4|1424687988397|1424689834443695938|
|stairsup|nexus4|1424687988598|1424689834645111953|
|stairsup|nexus4|1424687988804|1424689834846711074|
|stairsup|ne

In [ ]:
spltrans.stop()

In [ ]:
anotherStrm = streamdf.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time", "stairs")\
.writeStream\
.queryName("sTable")\
.format("memory")\
.outputMode("append")\
.start()

In [ ]:
spark.sql("select * from sTable").show()

+--------+------+-------------+-------------------+------+
|      gt| model| arrival_time|      creation_time|stairs|
+--------+------+-------------+-------------------+------+
|stairsup|nexus4|1424687983725|1424689829780924966|  true|
|stairsup|nexus4|1424687984011|1424687982020381820|  true|
|stairsup|nexus4|1424687984413|1424687982421559529|  true|
|stairsup|nexus4|1424687984818|1424687982827412800|  true|
|stairsup|nexus4|1424687985223|1424687983227571661|  true|
|stairsup|nexus4|1424687985623|1424687983630373175|  true|
|stairsup|nexus4|1424687986026|1424687984033235724|  true|
|stairsup|nexus4|1424687986428|1424687984435884650|  true|
|stairsup|nexus4|1424687986833|1424687984840816323|  true|
|stairsup|nexus4|1424687987240|1424687985246761147|  true|
|stairsup|nexus4|1424687987635|1424687985644600852|  true|
|stairsup|nexus4|1424687987994|1424689834040863906|  true|
|stairsup|nexus4|1424687988200|1424687986208748802|  true|
|stairsup|nexus4|1424687988397|1424689834443695938|  tru

In [ ]:
anotherStrm.stop()

In [ ]:
spark.streams.active

[]